In [1]:
#!/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/bin/python


#Standard Header used on the projects
# %%

dataSize = 'big'  # 'small'

#first the major packages used for math and graphing
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler

import os as os
import random
import multiprocessing
from joblib import Parallel, delayed

import pywt
from pywt._extensions._pywt import (DiscreteContinuousWavelet, ContinuousWavelet,
                                Wavelet, _check_dtype)
from pywt._functions import integrate_wavelet, scale2frequency
from time import time as ti
import datetime

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import  EarlyStopping
 
import cv2
from sklearn.model_selection import train_test_split
#import tensorflow.keras_metrics as km
  
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix

import platform

HostName = platform.node()

if HostName == "Server":
    Computer = "Desktop"   
elif HostName[-6:] == 'wm.edu':
    Computer = "SciClone"
elif HostName == "SchoolLaptop":
    Computer = "LinLap"
elif HostName == "WTC-TAB-512":
    Computer = "PortLap"
else:
    Computer = "WinLap"

if Computer == "SciClone":
    location = '/sciclone/home20/dchendrickson01/'
elif Computer == "WinLap":
    location = 'C:\\Data\\'
elif Computer == "Desktop":
    location = "E:\\Backups\\Dan\\CraneData\\"
elif Computer == "LinLap":
    location = '/home/dan/Output/'
    

if Computer ==  "SciClone":
    rootfolder = '/sciclone/home20/dchendrickson01/'
    if dataSize == 'big':
        folder = '/sciclone/scr10/dchendrickson01/CraneData/'
    else:
        folder = '/sciclone/data10/dchendrickson01/SmallCopy/'
elif Computer == "Desktop":
    rootfolder = location
    if dataSize == 'big':
        folder = 'G:\\CraneData\\'
    else:
        folder = rootfolder + "SmallCopy\\"
elif Computer =="WinLap":
    rootfolder = location
    folder = rootfolder + "SmallCopy\\"   
elif Computer == "LinLap":
    rootfolder = '/home/dan/Data/'
    folder = rootfolder + 'SmallCopy/'
    

scales = 500
#img_height , img_width = scales, 200
DoSomeFiles = False

SmoothType = 3  # 0 = none, 1 = rolling average, 2 = rolling StdDev
TrainEpochs = 3
WaveletToUse = 'beta'

num_cores = multiprocessing.cpu_count() -1
NumberOfFiles = num_cores - 2
GroupSize = NumberOfFiles

SensorPositonFile = rootfolder + 'SensorStatsSmall.csv'

if Computer == "SciClone" or Computer == "LinLap":
    SaveModelFolder = rootfolder + 'SavedModel2/'
else:
    SaveModelFolder = rootfolder + 'SavedModel\\'

files = os.listdir(folder)
if DoSomeFiles: files = random.sample(files,NumberOfFiles)

OutputVectors = np.genfromtxt(open(SensorPositonFile,'r'), delimiter=',',skip_header=1,dtype=int, missing_values=0)

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import CoreFunctions as cf

In [4]:
def resizeImage(FP):
    res = cv2.resize(FP, dsize=(int(np.shape(FP)[0]/2), int(np.shape(FP)[1]/6)), interpolation=cv2.INTER_CUBIC)

    return res

In [9]:
%%capture capt

def MakingTrainingSet(files):
    AllAccels = Parallel(n_jobs=num_cores)(delayed(cf.getAcceleration)(file) for file in files)
    Flattened = []
    for j in range(np.shape(AllAccels)[0]):
        if AllAccels[j][0] == False:
            print(j,AllAccels[j][1])
        else: 
            Flattened.append(AllAccels[j])
    print('Have Data')

    MetaData = []  #np.asarray([],dtype=object)
    DataOnlyMatrix = np.asarray([],dtype=object)
    for j in range(np.shape(AllAccels)[0]):
        if AllAccels[j][0] == False or np.shape(AllAccels[j][0][2])[0] != 60000:
            if AllAccels[j][1][4:9] =='Accel':
                print(j,AllAccels[j][1])
        else: 
            for k in range(3):
                MetaData.append([AllAccels[j][k][0], AllAccels[j][k][1], AllAccels[j][k][3], AllAccels[j][k][4]])
                if np.size(DataOnlyMatrix) == 0:
                        DataOnlyMatrix =np.matrix(AllAccels[j][k][2])
                else:
                        DataOnlyMatrix = np.concatenate((DataOnlyMatrix,np.matrix(AllAccels[j][k][2])),axis=0)

    MetaData = np.matrix(MetaData)

    AllAccels = cf.KalmanGroup(DataOnlyMatrix)

    del DataOnlyMatrix

    maxes = np.amax(AllAccels[:,500:], axis = 1)
    mins = np.amin(AllAccels[:,500:], axis = 1)

    Keep = np.zeros(mins.size)
    for i in range(mins.size):
        if i % 3 == 0:
            if maxes[i] > 0.01 and mins[i] < -0.01:
                Keep[i]=1
                Keep[i+1]=1
                Keep[i+2]=1
                #print(i)


    Keep = np.array(Keep, dtype='bool')

    AllAccels = AllAccels[Keep,:]
    MetaData = MetaData[Keep,:]

    MotionsLeft = int(np.shape(AllAccels)[0]/3.0)

    AllFingers =  Parallel(n_jobs=num_cores)(delayed(cf.makeMatrixImages)([AllAccels[i*3],AllAccels[i*3+1],AllAccels[i*3+2]]) for i in range(MotionsLeft))
    del AllAccels

    print('Have fingerprints')

    ResultsSet = []
    for i in range(MotionsLeft):
        ResultsSet.append(np.asarray(cf.truthVector(MetaData[i*3,3])).flatten())
    
    del MetaData

    print('Data Parsed')

    SmallFingers =  Parallel(n_jobs=num_cores)(delayed(resizeImage)(FP) for FP in AllFingers)
    del AllFingers

    X_train, X_test, y_train, y_test = train_test_split(SmallFingers, ResultsSet, test_size=0.20, shuffle=True, random_state=0)
    
    return X_train, X_test, y_train, y_test

In [10]:
GroupSize = NumberOfFiles

In [11]:


fCount = len(files)
GroupsLeft = int(fCount/GroupSize) + 1

SplitRatio = 1/(GroupsLeft)

RemainingFiles, GroupFiles, x,y = train_test_split(files, range(len(files)), test_size=SplitRatio, shuffle=True, random_state=0)

GroupsLeft -=1

X_train, X_test, y_train, y_test = MakingTrainingSet(GroupFiles)

Have Data


/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could

Have fingerprints
Data Parsed


In [12]:
img_width = np.shape(X_train)[1]
img_height = np.shape(X_train)[2]

In [13]:
model = Sequential([
    layers.Rescaling(1./1, input_shape=(img_width, img_height, 3)),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(np.shape(y_train)[1])
    ])
 
opt = tf.keras.optimizers.SGD(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
 
earlystop = EarlyStopping(patience=7)   
callbacks = [earlystop]

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 10000, 250, 3)     0         
                                                                 
 conv2d (Conv2D)             (None, 10000, 250, 32)    896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 5000, 125, 32)    0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 5000, 125, 32)     9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 2500, 62, 32)     0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 2500, 62, 64)      1

2022-07-01 09:55:15.003158: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-01 09:55:15.565608: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15389 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:04:00.0, compute capability: 6.0


In [14]:
y_train = np.matrix(y_train)

In [15]:
X_train = np.asarray(X_train)

In [16]:
history = model.fit(x = X_train, y = y_train, epochs=3, batch_size = 8 , shuffle=False, validation_split=0.25, callbacks=callbacks)

Epoch 1/3


2022-07-01 09:55:18.945295: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8401


3/3 [==============================] - 9s 796ms/step - loss: 6.9027 - accuracy: 0.2917 - val_loss: 2.4101 - val_accuracy: 0.1250
Epoch 2/3
3/3 [==============================] - 1s 495ms/step - loss: 5.4468 - accuracy: 0.3333 - val_loss: 2.2480 - val_accuracy: 0.1250
Epoch 3/3
3/3 [==============================] - 1s 479ms/step - loss: 6.0567 - accuracy: 0.2917 - val_loss: 4.1257 - val_accuracy: 0.1250


In [17]:
model.save(SaveModelFolder)

2022-07-01 09:55:29.236993: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /sciclone/home20/dchendrickson01/SavedModel2/assets


In [18]:

i = 1
while GroupsLeft > 0:
    SplitRatio = 1/(GroupsLeft)

    RemainingFiles, GroupFiles, x,y = train_test_split(RemainingFiles, range(len(RemainingFiles)), test_size=SplitRatio, shuffle=True, random_state=0)

    GroupsLeft -=1

    X_train, X_test, y_train, y_test = MakingTrainingSet(GroupFiles)
    #saver.restore('model.ckpt')
    if np.shape(X_train)[0] >= 4:
        y_train = np.matrix(y_train)
        X_train = np.asarray(X_train)
        
        history = model.fit(x = X_train, y = y_train, epochs=1, batch_size = 8 , shuffle=False, validation_split=0.25, callbacks=callbacks)
        if i%5 == 0:
            model.save(SaveModelFolder)
        i+=1
        print(i)
        #saver.save(sess,'model.ckpt')

Have Data


/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could

Have fingerprints
Data Parsed
3/3 [==============================] - 5s 2s/step - loss: 10.0107 - accuracy: 0.1905 - val_loss: 8.1946 - val_accuracy: 0.1250
2
Have Data


/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could

Have fingerprints
Data Parsed
3/3 [==============================] - 4s 2s/step - loss: 11.6408 - accuracy: 0.2222 - val_loss: 10.7955 - val_accuracy: 0.1667
3
Have Data


...

/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


Have fingerprints
Data Parsed
3/3 [==============================] - 2s 599ms/step - loss: 8.4741 - accuracy: 0.1429 - val_loss: 6.0698 - val_accuracy: 0.3750
5
Have Data


/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


Have fingerprints
Data Parsed
3/3 [==============================] - 1s 520ms/step - loss: 4.5230 - accuracy: 0.0556 - val_loss: 1.1921e-07 - val_accuracy: 0.1429
INFO:tensorflow:Assets written to: /sciclone/home20/dchendrickson01/SavedModel2/assets
6
Have Data
Have fingerprints
Data Parsed
3/3 [==============================] - 2s 815ms/step - loss: 6.1165 - accuracy: 0.2083 - val_loss: 5.3727 - val_accuracy: 0.1111
7
Have Data
Have fingerprints
Data Parsed
3/3 [==============================] - 4s 2s/step - loss: 4.9055 - accuracy: 0.0435 - val_loss: 8.0590 - val_accuracy: 0.1250
8
Have Data
Have fingerprints
Data Parsed
3/3 [==============================] - 2s 567ms/step - loss: 9.2551 - accuracy: 0.1905 - val_loss: 4.6417 - val_accuracy: 0.0000e+00
9
Have Data
Have fingerprints
Data Parsed
3/3 [==============================] - 4s 2s/step - loss: 11.2827 - accuracy: 0.1500 - val_loss: 4.7430 - val_accuracy: 0.0000e+00
10
Have Data
Have fingerprints
Data Parsed
3/3 [===============

ValueError: Some errors were detected !
    Line #47645 (got 10 columns instead of 5)
    Line #47646 (got 2 columns instead of 5)
    Line #47647 (got 8 columns instead of 5)
    Line #47648 (got 1 columns instead of 5)
    Line #47649 (got 2 columns instead of 5)
    Line #47650 (got 9 columns instead of 5)
    Line #47651 (got 3 columns instead of 5)
    Line #47652 (got 3 columns instead of 5)
    Line #47654 (got 4 columns instead of 5)
    Line #47655 (got 4 columns instead of 5)
    Line #47656 (got 2 columns instead of 5)
    Line #47657 (got 10 columns instead of 5)
    Line #47658 (got 4 columns instead of 5)
    Line #47659 (got 2 columns instead of 5)
    Line #47660 (got 3 columns instead of 5)
    Line #47661 (got 7 columns instead of 5)
    Line #47662 (got 6 columns instead of 5)
    Line #47664 (got 6 columns instead of 5)
    Line #47665 (got 2 columns instead of 5)
    Line #47666 (got 17 columns instead of 5)
    Line #47667 (got 3 columns instead of 5)
    Line #47

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig(rootfolder + 'ModelAccuracy.png')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig(rootfolder + 'ModelLoss.png')
plt.show()